In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
start_index = 20
end_index = 23
cam_name = 'cam2'

In [3]:
from services.label_service import get_map_not_zero_in_sphere

R = 10
map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, R, False)

map_size1=373853
map_size2=93297
map_size3=47374


In [4]:
from services.label_service import build_o3d_voxel_pcd

voxel_size = 0.2
voxel_pcd, voxel_points2instances, voxel_trace = build_o3d_voxel_pcd(map, points2instances, start_index, end_index, voxel_size)

In [5]:
voxel_pcd

PointCloud with 1934 points.

In [6]:
voxel_points2instances.shape

(1934, 6)

In [7]:
# import open3d as o3d
# from utils.pcd_utils import color_pcd_by_labels

# voxel_pcd_colored = color_pcd_by_labels(voxel_pcd, voxel_points2instances[:, 4])
# o3d.visualization.draw_geometries([voxel_pcd_colored])

In [8]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

voxel_points = np.asarray(voxel_pcd.points)
spatial_distance = cdist(voxel_points, voxel_points)

print("len(voxel_pcd.points) = {}".format(len(voxel_points)))
print("len(voxel_points2instances) = {}".format(len(voxel_points2instances)))

dist_voxel_points, masks = sam_label_distance(voxel_points2instances, spatial_distance, 2, 10)

len(voxel_pcd.points) = 1934
len(voxel_points2instances) = 1934


In [9]:
dist_voxel_points.shape

(1934, 1934)

In [10]:
from utils.distances_utils import remove_isolated_points

dist, points = remove_isolated_points(dist_voxel_points, voxel_points)

In [11]:
dist.shape

(1933, 1933)

In [12]:
from services.normalized_cut_service import normalized_cut

T = 0.5
eigenval = 100
clusters = normalized_cut(dist, np.array([i for i in range(len(points))], dtype=int), T, eigenval)

ev = [ 3.77964473e-01 -2.50896838e-21 -9.58473134e-19 ... -1.12090715e-18
 -3.34624530e-19 -3.10932438e-19]
mcut = 1.0395988821013121e-12
ev = [ 0.          0.84976381 -0.16234326  0.13466089 -0.31085999 -0.20036145
 -0.31085999]
mcut = 0.875


/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


ev = [0.00047647 0.00047647 0.00047647 ... 0.02343947 0.01071773 0.00587954]
mcut = 1.947639757277709e-07
ev = [0.00047647 0.00047647 0.00047647 ... 0.02343947 0.01071773 0.00587954]
mcut = 0.0024836308855239455
ev = [0.0004764  0.0004764  0.0004764  ... 0.02343947 0.01071773 0.00587954]
mcut = 0.02249758095036763
ev = [ 4.77600472e-04  4.77600486e-04  4.77600503e-04 ... -1.18226497e-05
 -5.40615916e-06 -2.96570901e-06]
mcut = 5.141770164824533e-09
ev = [-0.0004764  -0.0004764  -0.0004764  ... -0.02343947 -0.01071773
 -0.00587954]
mcut = 0.31927179325243893
ev = [ 0.00000000e+00  6.93889390e-18  0.00000000e+00 -9.44870027e-02
 -9.85825637e-02 -6.93889390e-18 -1.84486799e-02 -5.81458940e-03
 -9.96500976e-02  0.00000000e+00  6.93889390e-18  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -9.85824024e-02  0.00000000e+00
  0.00000000e+00  6.93889390e-18 -9.85825084e-02 -9.85825785e-02
 -6.93889390e-18 -2.72045332e-02  0.00000000e+00 -3.68579389e-02
 -2.45764724e-02 -9.96501626e-02 -1.07114

In [13]:
len(clusters)

41

In [14]:
from utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_clustered = color_pcd_by_clusters_and_voxels(map, voxel_pcd, voxel_trace, clusters)

In [16]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [17]:
executed_id = 2
name = "labeled_clouds/clouds_{}_{}_R{}_voxel{}_T{}_eigenval{}_execute_id{}".format(start_index, end_index, R, voxel_size, T, eigenval, executed_id)
filename = name.replace(".", "l") + ".pcd"

o3d.io.write_point_cloud(filename, pcd_clustered)

True

In [18]:
import open3d as o3d

pcd_read = o3d.io.read_point_cloud("labeled_clouds/clouds_20_23_R10_voxel0l2_T0l5_eigenval100_execute_id2.pcd")
o3d.visualization.draw_geometries([pcd_read])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
